In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
dfs = []
# for path in sorted(Path('../out/230122/cfs/').glob('*')):
# for path in sorted(Path('../out/230131/').glob('*')):
for path in sorted(Path('../out/230206/cfs/').glob('*')):
    if '.DS_Store' in str(path):
        continue
    df = pd.read_csv(path, delimiter=';', header=None)
    dfs.append(df)

In [4]:
df = pd.concat(dfs).reset_index()
df.columns = ['cf_idx', 'name', 'method', 'gci_weight', 'n_clusters', 'n_buses', 'n_lines', 'initial_load',
              'initial_line', 'lost_load', 'n_iterations', 'n_line_failures', 'n_gen_adjustments', 'n_alive_buses', 'n_final_components']
df = df[df['gci_weight'].isin([' no_gci', ' neg_weight'])]

In [8]:
pd.set_option('display.max_rows', 1000)

df[df['method'].isin([' original', ' warm_start_maximum_congestion', ' single_stage_power_flow_disruption', ' two_stage_maximum_congestion'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_8300/3328504586.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[df['method'].isin([' original', ' warm_start_maximum_congestion', ' single_stage_power_flow_disruption', ' two_stage_maximum_congestion'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()


cf_idx  \
name                       method                              gci_weight  n_clusters           
pglib_opf_case118_ieee      original                            no_gci     1             92.5   
                            warm_start_maximum_congestion       neg_weight 2             86.5   
                                                                           3             86.0   
pglib_opf_case162_ieee_dtc  original                            no_gci     1            141.5   
                            single_stage_power_flow_disruption  neg_weight 2            128.0   
                                                                           3            129.0   
                            two_stage_maximum_congestion        neg_weight 2            127.5   
                                                                           3            128.5   
                                                                           4            124.5   
                            warm_start_maximum_congestion       neg_weight 4            119.0   
pglib_opf_case179_goc       original                            no_gci     1            131.0   
                            single_stage_power_flow_disruption  neg_weight 2            129.0   
                                                                           3            126.5   
                                                                           4            125.5   
                            two_stage_maximum_congestion        neg_weight 2            129.0   
                                                                           3            126.5   
                                                                           4            125.5   
                            warm_start_maximum_congestion       neg_weight 2            128.0   
                                                                           3            123.0   
                                                                           4            122.0   
pglib_opf_case1888_rte      original                            no_gci     1           1265.0   
                            single_stage_power_flow_disruption  neg_weight 2           1253.5   
                                                                           3           1248.5   
                                                                           4           1237.0   
                            two_stage_maximum_congestion        neg_weight 2           1253.5   
                                                                           3           1248.5   
                                                                           4           1237.0   
                            warm_start_maximum_congestion       neg_weight 2           1253.5   
                                                                           3           1248.5   
                                                                           4           1237.0   
pglib_opf_case2000_goc      single_stage_power_flow_disruption  neg_weight 2           1801.0   
                                                                           3           1795.0   
                                                                           4           1790.5   
                            two_stage_maximum_congestion        neg_weight 2           1798.5   
                                                                           3           1791.0   
                                                                           4           1786.5   
                            warm_start_maximum_congestion       neg_weight 2           1801.0   
                                                                           3           1795.0   
                                                                           4           1790.5   
pglib_opf_case2848_rte      original                            no_gci     1           1887.5   
             

In [35]:
pd.set_option('display.max_rows', 1000)

df[df['method'].isin([' original', ' warm_start_maximum_congestion'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_1744/3737983804.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[df['method'].isin([' original', ' warm_start_maximum_congestion'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()


cf_idx  \
name                       method                         gci_weight  n_clusters           
pglib_opf_case118_ieee      warm_start_maximum_congestion  neg_weight 2             87.0   
                                                                      3             85.5   
pglib_opf_case162_ieee_dtc  warm_start_maximum_congestion  neg_weight 2            114.0   
                                                                      3            122.5   
                                                                      4            122.5   
                                                                      5            117.0   
pglib_opf_case179_goc       warm_start_maximum_congestion  neg_weight 2            128.0   
                                                                      3            123.0   
pglib_opf_case2000_goc      warm_start_maximum_congestion  neg_weight 2           1798.5   
                                                                      3           1791.0   
                                                                      4           1786.5   
pglib_opf_case300_ieee      warm_start_maximum_congestion  neg_weight 2            193.0   
pglib_opf_case30_ieee       warm_start_maximum_congestion  neg_weight 2             18.5   
                                                                      3             17.0   
                                                                      4             17.5   
                                                                      5             18.0   
pglib_opf_case39_epri       warm_start_maximum_congestion  neg_weight 2             21.5   
                                                                      3             21.5   
                                                                      4             21.0   
                                                                      5             21.0   
pglib_opf_case500_goc       warm_start_maximum_congestion  neg_weight 2            355.5   
                                                                      3            353.0   
                                                                      4            349.5   
                                                                      5            348.0   
pglib_opf_case57_ieee       warm_start_maximum_congestion  neg_weight 2             33.5   
                                                                      3             34.0   
                                                                      4             31.5   
                                                                      5             31.0   

                                                                                  n_buses  \
name                       method                         gci_weight  n_clusters            
pglib_opf_case118_ieee      warm_start_maximum_congestion  neg_weight 2             118.0   
                                                                      3             118.0   
pglib_opf_case162_ieee_dtc  warm_start_maximum_congestion  neg_weight 2             162.0   
                                                                      3             162.0   
                                                                      4             162.0   
                                                                      5             162.0   
pglib_opf_case179_goc       warm_start_maximum_congestion  neg_weight 2             179.0   
                                                                      3             179.0   
pglib_opf_case2000_goc      warm_start_maximum_congestion  neg_weight 2            2000.0   
                                                                      3            2000.0   
                                                                      4            2000.0   
pglib_opf_case300_ieee      warm_start_maximum_congestion  neg_weight 2             300.0   
pglib_opf_case30_ie

In [21]:
pd.set_option('display.max_rows', 1000)

df[df['method'].isin([' original', ' single_stage_maximum_congestion'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_1744/819664984.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[df['method'].isin([' original', ' single_stage_maximum_congestion'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()


,,,,cf_idx,n_buses,n_lines,initial_load,lost_load,n_iterations,n_line_failures,n_gen_adjustments,n_alive_buses,n_final_components
name,method,gci_weight,n_clusters,,,,,,,,,,


In [6]:
pd.set_option('display.max_rows', 1000)

df[df['method'].isin([' original', ' two_stage_maximum_congestion'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_1744/1227989148.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[df['method'].isin([' original', ' two_stage_maximum_congestion'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()


cf_idx  \
name                       method                        gci_weight  n_clusters           
pglib_opf_case118_ieee      original                      no_gci     1             92.5   
pglib_opf_case162_ieee_dtc  original                      no_gci     1            141.5   
                            two_stage_maximum_congestion  neg_weight 2            127.5   
                                                                     3            128.5   
                                                                     4            124.5   
                                                                     5            122.0   
pglib_opf_case179_goc       original                      no_gci     1            131.0   
                            two_stage_maximum_congestion  neg_weight 2            129.0   
                                                                     3            126.5   
                                                                     4            125.5   
                                                                     5            125.5   
pglib_opf_case1888_rte      original                      no_gci     1           1265.0   
                            two_stage_maximum_congestion  neg_weight 2           1253.5   
                                                                     3           1248.5   
                                                                     4           1237.0   
                                                                     5           1225.0   
pglib_opf_case1951_rte      original                      no_gci     1           1297.5   
                            two_stage_maximum_congestion  neg_weight 2           1285.0   
pglib_opf_case2000_goc      two_stage_maximum_congestion  neg_weight 2           1798.5   
                                                                     3           1791.0   
                                                                     4           1786.5   
pglib_opf_case200_activ     original                      no_gci     1            122.0   
                            two_stage_maximum_congestion  neg_weight 2            118.0   
                                                                     3            117.5   
                                                                     4            117.0   
                                                                     5            115.5   
pglib_opf_case240_pserc     original                      no_gci     1            223.5   
pglib_opf_case2848_rte      original                      no_gci     1           1887.5   
                            two_stage_maximum_congestion  neg_weight 2           1873.0   
                                                                     3           1865.0   
                                                                     4           1856.5   
pglib_opf_case300_ieee      original                      no_gci     1            205.0   
                            two_stage_maximum_congestion  neg_weight 2            199.5   
                                                                     3            197.0   
                                                                     4            192.0   
                                                                     5            189.0   
pglib_opf_case30_ieee       original                      no_gci     1             20.0   
                            two_stage_maximum_congestion  neg_weight 5             18.5   
pglib_opf_case39_epri       original                      no_gci     1             22.5   
                            two_stage_maximum_congestion  neg_weight 2             21.5   
                                                                     3             21.5   
                                                                     4             21.0   
                                                                     5       

In [10]:
pd.set_option('display.max_rows', 1000)

df[df['method'].isin([' original', ' single_stage_power_flow_disruption'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_1744/2541204458.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[df['method'].isin([' original', ' single_stage_power_flow_disruption'])].groupby(['name', 'method', 'gci_weight', 'n_clusters']).mean()


cf_idx  \
name                       method                              gci_weight  n_clusters           
pglib_opf_case118_ieee      original                            no_gci     1             92.5   
pglib_opf_case162_ieee_dtc  original                            no_gci     1            141.5   
                            single_stage_power_flow_disruption  neg_weight 2            128.0   
                                                                           3            129.0   
                                                                           5            123.0   
pglib_opf_case179_goc       original                            no_gci     1            131.0   
                            single_stage_power_flow_disruption  neg_weight 2            129.0   
                                                                           3            126.5   
                                                                           4            125.5   
                                                                           5            125.5   
pglib_opf_case1888_rte      original                            no_gci     1           1265.0   
                            single_stage_power_flow_disruption  neg_weight 2           1253.5   
                                                                           3           1248.5   
                                                                           4           1237.0   
pglib_opf_case1951_rte      original                            no_gci     1           1297.5   
                            single_stage_power_flow_disruption  neg_weight 2           1285.0   
                                                                           3           1274.0   
                                                                           4           1269.5   
                                                                           5           1263.0   
pglib_opf_case2000_goc      single_stage_power_flow_disruption  neg_weight 2           1801.0   
                                                                           3           1795.0   
                                                                           4           1790.5   
pglib_opf_case200_activ     original                            no_gci     1            122.0   
                            single_stage_power_flow_disruption  neg_weight 2            118.0   
                                                                           3            117.5   
                                                                           4            117.0   
                                                                           5            116.0   
pglib_opf_case240_pserc     original                            no_gci     1            223.5   
pglib_opf_case2848_rte      original                            no_gci     1           1887.5   
                            single_stage_power_flow_disruption  neg_weight 2           1872.5   
                                                                           3           1864.5   
                                                                           4           1855.0   
                                                                           5           1845.0   
pglib_opf_case300_ieee      original                            no_gci     1            205.0   
                            single_stage_power_flow_disruption  neg_weight 2            199.5   
                                                                           3            197.0   
                                                                           4            192.0   
                                                                           5            191.5   
pglib_opf_case30_ieee       original                            no_gci     1             20.0   
                            single_stage_power_flow_disruption  neg_weight 4             17.5   
             

In [5]:
# dfs = []
# for path in sorted(Path('../nresults/cfd/').glob('*/**/*')):
#     if '.DS_Store' in str(path):
#         continue
#     df = pd.read_csv(path, delimiter=';', header=None)
#     dfs.append(df)

# #     print('\n')
            
# #     fig, ax = plt.subplots(figsize=[20, 8])
# #     ax.boxplot([mat[:, 0] for mat in stats.values()])
# #     ax.set_xticklabels([k.split('-', 1)[1] for k in stats.keys()])
# #     plt.xticks(rotation=90)
# #     plt.title(f'{case}')
    
# #     og = [mat[:, 0] for name, mat in stats.items() if 'og' in name]
# #     tp = [mat[:, 0] for name, mat in stats.items() if 'tp' in name and not 'opf' in name]
# #     tp = [mat[:, 0] for name, mat in stats.items() if 'tp' in name and 'opf' in name]

In [27]:
# df = pd.concat(dfs).reset_index()
# df.columns = ['cf_idx', 'case', 'name', 'n_buses', 'n_lines', 'n_clusters', 'initial_line', 'lost_load', 'n_line_failures', 'n_gen_adjustments', 'n_alive_buses', 'n_final_components']

# splitweights = pd.DataFrame(df['name'].apply(lambda s: s.split('-')[-2:]).values.tolist(), columns=['split', 'weight'])

# df[['split', 'weight']] = splitweights
# # df = df.loc[df['weight'] == 'neg_weight']

In [9]:
# pd.set_option('display.max_rows', None)  # or 1000

# df.groupby(['case', 'weight', 'split']).mean().round(1)

In [8]:
# def get_stats(vals):
#     """
#     Mean load shed, mean n_line failures, mean alive buses, mean components
#     """
#     return [np.mean(vals[:, 0]), np.mean(vals[:, 1]), np.mean(vals[:, 3]), np.mean(vals[:, 4])]

In [9]:
# dfdata = []

# for case in cases:
#     stats = {}
    
#     for path in sorted(Path('../res-cf/').glob(case + '*'), key=lambda x: str(x.stem)[-2:]):
#         data = np.genfromtxt(path, delimiter=';')        
#         stats[path.stem] = data
    
#     # Lost load; but alos get n_line failures and n_final_components
#     og = [mat for name, mat in stats.items() if 'og' in name][0]
#     dfdata.append([case, 1] + get_stats(og))
    
#     for k in range(2, 5):
#         opf = [mat for name, mat in stats.items() if 'tp' in name and 'opf' in name and str(k) in name]
   
#         if opf:
#             dfdata.append([case, k] + get_stats(opf[0]))

NameError: name 'cases' is not defined

In [ ]:
# df = pd.DataFrame(dfdata, columns=['case', 'k', 'lost load', 'line failures', 'alive buses', 'final components'])
# df['size'] = df['case'].apply(lambda x: int(x.split('_')[0].lstrip('case')[:4]))
# df = df[df['size'] >= 100]

In [ ]:
# grouped = df.sort_values(['size', 'k']).groupby(['case', 'k'])['lost load', 'line failures', 'alive buses', 'final components'].sum().round(0).astype(int)
# print(grouped.to_latex())
